In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc

plt.rcParams["figure.figsize"] = [12, 12]
# If you have problems with latex at matplotlib just comment next two lines, this might help
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
 

def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1 - d2) / 2, ylim[1] + (d1 - d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1 - d2) / 2, xlim[1] - (d1 - d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1 - d2) / 2, ylim[1] + (d1 - d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1 - d2) / 2, xlim[1] - (d1 - d2) / 2))


In [3]:
# 2.1: f, grad
def func(x):
    return np.e ** (x[0] + 3 * x[1]) + np.e ** (x[0] - 3 * x[1]) + np.e ** (-x[0])


def f_grad(x):
    return np.array([np.e ** (x[0] + 3 * x[1]) + np.e ** (x[0] - 3 * x[1]) - np.e ** (-x[0]),
                     3 * (np.e ** (x[0] + 3 * x[1]) - np.e ** (x[0] - 3 * x[1]))])


min_point = [-np.log(2) / 2, 0]

In [4]:
# 2.2: estimation of Lipschitz constant
def get_lipschitz_grid(x_range, y_range):
    L = 0
    delta = 0.1
    x = np.arange(x_range[0], x_range[1], delta)
    y = np.arange(y_range[0], y_range[1], delta)
    X, Y = np.meshgrid(x, y)
    ZX = np.zeros_like(X)
    ZY = np.zeros_like(X)
    # print(X.shape, Y.shape)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            grad = f_grad([X[i][j], Y[i][j]])
            ZX[i][j] = grad[0]
            ZY[i][j] = grad[1]
    # print(range(X.shape[0]))
    # print(range(X.shape[1]))
    for i1 in range(X.shape[0]):
        for j1 in range(X.shape[1]):
            for i2 in range(X.shape[0]):
                for j2 in range(X.shape[1]):
                    x1 = X[i1][j1]
                    y1 = Y[i1][j1]
                    x2 = X[i2][j2]
                    y2 = Y[i2][j2]  
                    # print("Point1: ", x1, y1)
                    # print("Point2: ", x2, y2)
                    norm_points = np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))
                    norm_f = np.linalg.norm(np.array([ZX[i1][j1], ZY[i1][j1]]) - np.array([ZX[i2][j2], ZY[i2][j2]]))
                    # print("Norm point: ", norm_points)
                    # print("Norm f: ", norm_f)
                    if norm_points == 0:
                        continue
                    new_L = norm_f / norm_points
                    # print("New L: ", new_L)
                    L = max(L, new_L)
    return L

In [5]:
# 2.2: estimation of Lipschitz constant
def get_lipschitz_random(x_range, y_range):
    L = 0
    iterations_number = 100000
    for i in range(iterations_number):
        rand = np.random.random([4])
        x1 = x_range[0] + (x_range[1] - x_range[0]) * rand[0]
        y1 = y_range[0] + (y_range[1] - y_range[0]) * rand[1]
        x2 = x_range[0] + (x_range[1] - x_range[0]) * rand[2]
        y2 = y_range[0] + (y_range[1] - y_range[0]) * rand[3]
        # print("Point1: ", x1, y1)
        # print("Point2: ", x2, y2)
        norm_points = np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))
        norm_f = np.linalg.norm(np.array(f_grad([x1, y1])) - np.array(f_grad([x2, y2])))
        # print("Norm point: ", norm_points)
        # print("Norm f: ", norm_f)
        if norm_points == 0:
            continue
        new_L = norm_f / norm_points
        # print("New L: ", new_L)
        L = max(L, new_L)
    return L

In [6]:
# 2.2: estimation of Lipschitz constant
def get_lipschitz(x_range, y_range):
    return max(get_lipschitz_grid(x_range, y_range), get_lipschitz_random(x_range, y_range))

In [7]:
# 2.2: estimation of Lipschitz constant
L_constant = get_lipschitz_grid([-1, 1], [-1, 1])
print("Grid: ", L_constant)

Grid:  405.98947162419614


In [8]:
# 2.2: estimation of Lipschitz constant
L_constant = get_lipschitz_random([-1, 1], [-1, 1])
print("Random:", L_constant)

Random: 404.4369191118598


In [9]:
# 2.2: estimation of Lipschitz constant
L_constant = get_lipschitz([-1, 1], [-1, 1])
print("Lipschitz constant approximately equals ", L_constant)

Lipschitz constant approximately equals  442.8626287972052


In [12]:
# 2.3
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


def animate_trajectory(trajectory):
    fig, ax = plt.subplots()
    n = len(trajectory)

    def step(t):
        ax.cla()
        ax.plot([min_point[0]], [min_point[1]], 'o', color='green')
        # Level contours
        delta = 0.025
        x = np.arange(-2, 4, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        # print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [4, 8, 12], colors=['blue', 'purple', 'red'])

        ax.plot([u[0] for u in trajectory[:t]], [u[1] for u in trajectory[:t]], color='black')
        ax.plot([u[0] for u in trajectory[:t]], [u[1] for u in trajectory[:t]], 'o', color='black')
        
        fix_scaling(ax)
        ax.axis('off')

    return FuncAnimation(fig, step, frames=range(n), interval=600)

In [32]:
# 2.3: Nesterov's method
M = 1.3 * get_lipschitz_grid([-1, 1], [-1, 1]) # "1.3 *" to be sure
alpha = 1 / M
trajectory_nesterov = []
x_start = np.array([1, 1])
trajectory_nesterov.append(x_start.copy())
cur_x = x_start.copy()
cur_y = x_start.copy()
cur_alpha = 0.9
iterations_number = 50
for i in range(iterations_number):
    cur_x_prev = cur_x
    cur_x = cur_y - alpha * f_grad(cur_y)
    cur_alpha_prev = cur_alpha
    cur_alpha = 0.5 * (cur_alpha * np.sqrt(cur_alpha ** 2 + 4) - cur_alpha ** 2)
    beta = cur_alpha_prev * (1 - cur_alpha_prev) / (cur_alpha_prev ** 2 + cur_alpha)
    cur_y = cur_x + beta * (cur_x - cur_x_prev)
    trajectory_nesterov.append(cur_x.copy())

base_animation = animate_trajectory(trajectory_nesterov)
HTML(base_animation.to_html5_video())

In [42]:
# 2.3: Gradient descent with optimal step
M = get_lipschitz_grid([-1, 1], [-1, 1])
alpha = 2.0 / M
trajectory_opt_step = []
x_start = np.array([1, 1])
trajectory_opt_step.append(x_start.copy())
cur_x = x_start.copy()
iterations_number = 250
for i in range(iterations_number):
    cur_x = cur_x - alpha * f_grad(cur_x)
    trajectory_opt_step.append(cur_x.copy())
    
# print(trajectory_opt_step)
base_animation = animate_trajectory(trajectory_opt_step)
HTML(base_animation.to_html5_video())

/home/mikhail/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
